In [1]:
! pip install datasets transformers seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install spacy

In [2]:
from datasets import Dataset, ClassLabel, Sequence, load_dataset, load_metric
import numpy as np
import pandas as pd
from spacy import displacy
import transformers
from transformers import (AutoModelForTokenClassification, 
                          AutoTokenizer, 
                          DataCollatorForTokenClassification,
                          pipeline,
                          TrainingArguments, 
                          Trainer)

In [3]:
# confirm version > 4.11.0
print(transformers.__version__)

4.29.2


In [4]:
datasets = load_dataset("ade_corpus_v2", "Ade_corpus_v2_drug_dosage_relation")

  0%|          | 0/1 [00:00<?, ?it/s]

In [5]:
datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'drug', 'dosage', 'indexes'],
        num_rows: 279
    })
})

In [6]:
datasets["train"][0]

{'text': 'An episode of subacute encephalopathy after the infusion of a moderate dose of methotrexate (1500 mg/m2) (MTX) is reported in a young adult with metastastic gastric cancer.',
 'drug': 'methotrexate',
 'dosage': '1500 mg/m2',
 'indexes': {'drug': {'start_char': [79], 'end_char': [91]},
  'dosage': {'start_char': [93], 'end_char': [103]}}}

In [7]:
consolidated_dataset = {}

for row in datasets["train"]:
    if row["text"] in consolidated_dataset:
        consolidated_dataset[row["text"]]["drug_indices_start"].update(row["indexes"]["drug"]["start_char"])
        consolidated_dataset[row["text"]]["drug_indices_end"].update(row["indexes"]["drug"]["end_char"])
        consolidated_dataset[row["text"]]["dosage_indices_start"].update(row["indexes"]["dosage"]["start_char"])
        consolidated_dataset[row["text"]]["dosage_indices_end"].update(row["indexes"]["dosage"]["end_char"])
        consolidated_dataset[row["text"]]["drug"].append(row["drug"])
        consolidated_dataset[row["text"]]["dosage"].append(row["dosage"])
        
    else:
        consolidated_dataset[row["text"]] = {
            "text": row["text"],
            "drug": [row["drug"]],
            "dosage": [row["dosage"]],
            # use sets because the indices can repeat for various reasons
            "drug_indices_start": set(row["indexes"]["drug"]["start_char"]),
            "drug_indices_end": set(row["indexes"]["drug"]["end_char"]),
            "dosage_indices_start": set(row["indexes"]["dosage"]["start_char"]),
            "dosage_indices_end": set(row["indexes"]["dosage"]["end_char"])
        }

In [ ]:
consolidated_dataset

In [9]:
df = pd.DataFrame(list(consolidated_dataset.values()))
df.head()

text            drug  \
0  An episode of subacute encephalopathy after th...  [methotrexate]   
1  She continued to receive regular insulin 4 tim...       [insulin]   
2  A 5-month-old infant became lethargic and poor...   [brimonidine]   
3  The presented patient was treated with 200 mg ...           [TCA]   
4  Central nervous system manifestations of an ib...     [ibuprofen]   

              dosage drug_indices_start drug_indices_end dosage_indices_start  \
0       [1500 mg/m2]               {79}             {91}                 {93}   
1  [4 times per day]               {33}             {40}                 {41}   
2           [1 drop]               {86}             {97}                 {76}   
3           [200 mg]          {163, 46}        {49, 166}                 {39}   
4         [overdose]               {44}             {53}                 {54}   

  dosage_indices_end  
0              {103}  
1               {56}  
2               {82}  
3               {45}  
4               {62}

In [10]:
# since no spans overlap, we can sort to get 1:1 matched index spans
# note that sets don't preserve insertion order

df["drug_indices_start"] = df["drug_indices_start"].apply(list).apply(sorted)
df["drug_indices_end"] = df["drug_indices_end"].apply(list).apply(sorted)
df["dosage_indices_start"] = df["dosage_indices_start"].apply(list).apply(sorted)
df["dosage_indices_end"] = df["dosage_indices_end"].apply(list).apply(sorted)

In [11]:
df

text  \
0    An episode of subacute encephalopathy after th...   
1    She continued to receive regular insulin 4 tim...   
2    A 5-month-old infant became lethargic and poor...   
3    The presented patient was treated with 200 mg ...   
4    Central nervous system manifestations of an ib...   
..                                                 ...   
208  1. Changes in the plasma cortisol level were r...   
209  We report on three patients with acute schizop...   
210  A 65-year-old woman with angina pectoris prese...   
211  In a postural challenge test after administrat...   
212  High-dose phosphate treatment leads to hypokal...   

                       drug             dosage drug_indices_start  \
0            [methotrexate]       [1500 mg/m2]               [79]   
1                 [insulin]  [4 times per day]               [33]   
2             [brimonidine]           [1 drop]               [86]   
3                     [TCA]           [200 mg]          [46, 163]   
4               [ibuprofen]         [overdose]               [44]   
..                      ...                ...                ...   
208            [alprazolam]              [low]              [122]   
209            [olanzapine]       [20-25 mg/d]              [107]   
210  [isosorbide dinitrate]             [5 mg]               [94]   
211  [isosorbide dinitrate]             [5 mg]               [53]   
212             [phosphate]             [High]           [10, 58]   

    drug_indices_end dosage_indices_start dosage_indices_end  
0               [91]                 [93]              [103]  
1               [40]                 [41]               [56]  
2               [97]                 [76]               [82]  
3          [49, 166]                 [39]               [45]  
4               [53]                 [54]               [62]  
..               ...                  ...                ...  
208            [132]                [113]              [116]  
209            [117]                [119]              [129]  
210            [114]                [116]              [120]  
211             [73]                 [75]               [79]  
212         [19, 67]                  [0]                [4]  

[213 rows x 7 columns]

In [12]:
# save to JSON to then import into Dataset object
df.to_json("dataset.jsonl", orient="records", lines=True)

In [13]:
cons_dataset = load_dataset("json", data_files="dataset.jsonl")
cons_dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-322d288bfbc24d09/0.0.0/e347ab1c932092252e717ff3f949105a4dd28b27e842dd53157d2f72e276c2e4. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'drug', 'dosage', 'drug_indices_start', 'drug_indices_end', 'dosage_indices_start', 'dosage_indices_end'],
        num_rows: 213
    })
})

In [14]:
# no train-test provided, so we create our own
cons_dataset = cons_dataset["train"].train_test_split()

In [15]:
cons_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'drug', 'dosage', 'drug_indices_start', 'drug_indices_end', 'dosage_indices_start', 'dosage_indices_end'],
        num_rows: 159
    })
    test: Dataset({
        features: ['text', 'drug', 'dosage', 'drug_indices_start', 'drug_indices_end', 'dosage_indices_start', 'dosage_indices_end'],
        num_rows: 54
    })
})

In [16]:
label_list = ['O', 'B-DRUG', 'I-DRUG', 'B-DOSAGE', 'I-DOSAGE']

custom_seq = Sequence(feature=ClassLabel(num_classes=5, 
                                         names=label_list,
                                         names_file=None, id=None), length=-1, id=None)

cons_dataset["train"].features["ner_tags"] = custom_seq
cons_dataset["test"].features["ner_tags"] = custom_seq

In [17]:
model_name = "allenai/scibert_scivocab_uncased"

In [18]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [19]:
def generate_row_labels(row, verbose=False):
    """ Given a row from the consolidated `Ade_corpus_v2_drug_ade_relation` dataset, 
    generates BIO tags for drug and dosage entities. 
    
    """

    text = row["text"]

    labels = []
    label = "O"
    prefix = ""
    
    # while iterating through tokens, increment to traverse all drug and effect spans
    drug_index = 0
    dosage_index = 0
    
    tokens = tokenizer(text, return_offsets_mapping=True)

    for n in range(len(tokens["input_ids"])):
        offset_start, offset_end = tokens["offset_mapping"][n]

        # should only happen for [CLS] and [SEP]
        if offset_end - offset_start == 0:
            labels.append(-100)
            continue
        
        if drug_index < len(row["drug_indices_start"]) and offset_start == row["drug_indices_start"][drug_index]:
            label = "DRUG"
            prefix = "B-"

        elif dosage_index < len(row["dosage_indices_start"]) and offset_start == row["dosage_indices_start"][dosage_index]:
            label = "DOSAGE"
            prefix = "B-"
        
        labels.append(label_list.index(f"{prefix}{label}"))
            
        if drug_index < len(row["drug_indices_end"]) and offset_end == row["drug_indices_end"][drug_index]:
            label = "O"
            prefix = ""
            drug_index += 1
            
        elif dosage_index < len(row["dosage_indices_end"]) and offset_end == row["dosage_indices_end"][dosage_index]:
            label = "O"
            prefix = ""
            dosage_index += 1

        # need to transition "inside" if we just entered an entity
        if prefix == "B-":
            prefix = "I-"
    
    if verbose:
        print(f"{row}\n")
        orig = tokenizer.convert_ids_to_tokens(tokens["input_ids"])
        for n in range(len(labels)):
            print(orig[n], labels[n])
    tokens["labels"] = labels
    
    return tokens

In [20]:
# testing out...

generate_row_labels(cons_dataset["train"][2], verbose=True)

{'text': 'She was receiving phenytoin sodium 300 mg/day; carbamazepine 200 mg four times daily had been discontinued four days before admission because of leukopenia.', 'drug': ['carbamazepine', 'phenytoin sodium'], 'dosage': ['200 mg', '300 mg/day'], 'drug_indices_start': [18, 47], 'drug_indices_end': [34, 60], 'dosage_indices_start': [35, 61], 'dosage_indices_end': [45, 67]}

[CLS] -100
she 0
was 0
receiving 0
phen 1
##yt 2
##oin 2
sodium 2
300 3
mg 4
/ 4
day 4
; 0
carb 1
##ama 2
##ze 2
##pine 2
200 3
mg 4
four 0
times 0
daily 0
had 0
been 0
discontinued 0
four 0
days 0
before 0
admission 0
because 0
of 0
leuk 0
##openia 0
. 0
[SEP] -100


{'input_ids': [102, 2281, 241, 5484, 1952, 5571, 616, 5183, 4825, 1529, 1352, 2181, 1814, 2445, 5179, 1948, 14333, 369, 1529, 1379, 1838, 4122, 883, 528, 22221, 1379, 1976, 1548, 7512, 923, 131, 5133, 28565, 205, 103], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'offset_mapping': [(0, 0), (0, 3), (4, 7), (8, 17), (18, 22), (22, 24), (24, 27), (28, 34), (35, 38), (39, 41), (41, 42), (42, 45), (45, 46), (47, 51), (51, 54), (54, 56), (56, 60), (61, 64), (65, 67), (68, 72), (73, 78), (79, 84), (85, 88), (89, 93), (94, 106), (107, 111), (112, 116), (117, 123), (124, 133), (134, 141), (142, 144), (145, 149), (149, 155), (155, 156), (0, 0)], 'labels': [-100, 0, 0, 0, 1, 2, 2, 2, 3, 4, 4, 4, 0, 1, 2, 2, 2, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -100]}

In [21]:
labeled_dataset = cons_dataset.map(generate_row_labels)
labeled_dataset

Map:   0%|          | 0/159 [00:00<?, ? examples/s]

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'drug', 'dosage', 'drug_indices_start', 'drug_indices_end', 'dosage_indices_start', 'dosage_indices_end', 'input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'labels'],
        num_rows: 159
    })
    test: Dataset({
        features: ['text', 'drug', 'dosage', 'drug_indices_start', 'drug_indices_end', 'dosage_indices_start', 'dosage_indices_end', 'input_ids', 'token_type_ids', 'attention_mask', 'offset_mapping', 'labels'],
        num_rows: 54
    })
})

SciBERT Model Fine-Tuning

In [22]:
task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "allenai/scibert_scivocab_uncased"
batch_size = 16

In [23]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at allenai/scibert_scivocab_uncased were not used when initializing BertForTokenClassification: ['cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initi

In [24]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.05,
    logging_steps=1
)

In [26]:
data_collator = DataCollatorForTokenClassification(tokenizer)
data_collator

DataCollatorForTokenClassification(tokenizer=BertTokenizerFast(name_or_path='allenai/scibert_scivocab_uncased', vocab_size=31090, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True), padding=True, max_length=None, pad_to_multiple_of=None, label_pad_token_id=-100, return_tensors='pt')

In [27]:
metric = load_metric("seqeval")

<ipython-input-27-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [28]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [29]:
trainer = Trainer(
    model,
    args,
    train_dataset=labeled_dataset["train"],
    eval_dataset=labeled_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, 

)

In [30]:
trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=50, training_loss=0.4814326658844948, metrics={'train_runtime': 619.4537, 'train_samples_per_second': 1.283, 'train_steps_per_second': 0.081, 'total_flos': 25706275996830.0, 'train_loss': 0.4814326658844948, 'epoch': 5.0})

In [31]:
predictions, labels, _ = trainer.predict(labeled_dataset["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

{'DOSAGE': {'precision': 0.5423728813559322,
  'recall': 0.5614035087719298,
  'f1': 0.5517241379310345,
  'number': 57},
 'DRUG': {'precision': 0.8064516129032258,
  'recall': 0.7142857142857143,
  'f1': 0.7575757575757576,
  'number': 70},
 'overall_precision': 0.6776859504132231,
 'overall_recall': 0.6456692913385826,
 'overall_f1': 0.6612903225806451,
 'overall_accuracy': 0.9347705914870094}

Model Outputs

In [33]:
dosage_ner_model = pipeline(task="ner", model=model, tokenizer=tokenizer)

In [34]:
# something from our validation set
dosage_ner_model(labeled_dataset["test"][4]["text"])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


[{'entity': 'LABEL_0',
  'score': 0.9913867,
  'index': 1,
  'word': 'we',
  'start': 0,
  'end': 2},
 {'entity': 'LABEL_0',
  'score': 0.99314755,
  'index': 2,
  'word': 'report',
  'start': 3,
  'end': 9},
 {'entity': 'LABEL_0',
  'score': 0.99455404,
  'index': 3,
  'word': 'a',
  'start': 10,
  'end': 11},
 {'entity': 'LABEL_0',
  'score': 0.99397016,
  'index': 4,
  'word': 'case',
  'start': 12,
  'end': 16},
 {'entity': 'LABEL_0',
  'score': 0.9938605,
  'index': 5,
  'word': 'of',
  'start': 17,
  'end': 19},
 {'entity': 'LABEL_0',
  'score': 0.9556386,
  'index': 6,
  'word': 'unin',
  'start': 20,
  'end': 24},
 {'entity': 'LABEL_0',
  'score': 0.97233486,
  'index': 7,
  'word': '##tention',
  'start': 24,
  'end': 31},
 {'entity': 'LABEL_0',
  'score': 0.9750289,
  'index': 8,
  'word': '##al',
  'start': 31,
  'end': 33},
 {'entity': 'LABEL_0',
  'score': 0.6588931,
  'index': 9,
  'word': 'over',
  'start': 34,
  'end': 38},
 {'entity': 'LABEL_0',
  'score': 0.83493644,


In [42]:
def visualize_entities(sentence):
    tokens = dosage_ner_model(sentence)
    entities = []
    
    for token in tokens:
        label = int(token["entity"][-1])
        if label != 0:
            token["label"] = label_list[label]
            entities.append(token)
            # print(entities)
    
    params = [{"text": sentence,
               "ents": entities,
               "title": None}]
    
    print(params)
    html = displacy.render(params, style="ent", manual=True, options={
        "colors": {
                   "B-DRUG": "#f08080",
                   "I-DRUG": "#f08080",
                   "B-DOSAGE": "#9bddff",
                   "I-DOSAGE": "#9bddff",
               },
    })

In [43]:
examples = [
    "The patient was prescribed a daily dose of 500mg of aspirin to manage their pain.",
    "The doctor recommended taking 2 tablets of ibuprofen, each containing 200mg, every 6 hours.",
"The pharmacist dispensed a 10-day supply of amoxicillin, with each capsule containing 250mg.",
"The patient's medication regimen includes a weekly injection of 100mg of methotrexate.",
"The doctor adjusted the dosage of the antidepressant to 20mg per day to achieve better results.",
"The prescription specifies taking 1 tablet of paracetamol, 500mg, as needed for fever or pain.",
"The patient is currently on a low-dose regimen of 50mcg of levothyroxine for thyroid management.",
"The medication label advises a maximum daily dose of 10mg for the antihistamine allergy medication."
]

for example in examples:
    visualize_entities(example)
    print(f"{'*' * 50}\n")

[{'text': 'The patient was prescribed a daily dose of 500mg of aspirin to manage their pain.', 'ents': [{'entity': 'LABEL_4', 'score': 0.2768554, 'index': 9, 'word': '500', 'start': 43, 'end': 46, 'label': 'I-DOSAGE'}, {'entity': 'LABEL_4', 'score': 0.52765775, 'index': 10, 'word': '##mg', 'start': 46, 'end': 48, 'label': 'I-DOSAGE'}], 'title': None}]
**************************************************

[{'text': 'The doctor recommended taking 2 tablets of ibuprofen, each containing 200mg, every 6 hours.', 'ents': [{'entity': 'LABEL_1', 'score': 0.5523564, 'index': 8, 'word': 'ib', 'start': 43, 'end': 45, 'label': 'B-DRUG'}, {'entity': 'LABEL_2', 'score': 0.72151303, 'index': 9, 'word': '##up', 'start': 45, 'end': 47, 'label': 'I-DRUG'}, {'entity': 'LABEL_2', 'score': 0.7737116, 'index': 10, 'word': '##rof', 'start': 47, 'end': 50, 'label': 'I-DRUG'}, {'entity': 'LABEL_2', 'score': 0.75760174, 'index': 11, 'word': '##en', 'start': 50, 'end': 52, 'label': 'I-DRUG'}, {'entity': 'LABEL_3',

In [39]:
dosage_ner_model("The patient was prescribed a daily dose of 500mg of aspirin to manage their pain.")

[{'entity': 'LABEL_0',
  'score': 0.98782015,
  'index': 1,
  'word': 'the',
  'start': 0,
  'end': 3},
 {'entity': 'LABEL_0',
  'score': 0.98816246,
  'index': 2,
  'word': 'patient',
  'start': 4,
  'end': 11},
 {'entity': 'LABEL_0',
  'score': 0.9822556,
  'index': 3,
  'word': 'was',
  'start': 12,
  'end': 15},
 {'entity': 'LABEL_0',
  'score': 0.96842736,
  'index': 4,
  'word': 'prescribed',
  'start': 16,
  'end': 26},
 {'entity': 'LABEL_0',
  'score': 0.8910099,
  'index': 5,
  'word': 'a',
  'start': 27,
  'end': 28},
 {'entity': 'LABEL_0',
  'score': 0.7468781,
  'index': 6,
  'word': 'daily',
  'start': 29,
  'end': 34},
 {'entity': 'LABEL_0',
  'score': 0.76609033,
  'index': 7,
  'word': 'dose',
  'start': 35,
  'end': 39},
 {'entity': 'LABEL_0',
  'score': 0.9501829,
  'index': 8,
  'word': 'of',
  'start': 40,
  'end': 42},
 {'entity': 'LABEL_4',
  'score': 0.2768554,
  'index': 9,
  'word': '500',
  'start': 43,
  'end': 46},
 {'entity': 'LABEL_4',
  'score': 0.5276577